In [1]:
import os
os.chdir('..')

In [12]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1992

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [13]:
import utils
from keras.models import Sequential
from keras.layers import Dense, ReLU
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D, AveragePooling1D
import tensorflow as tf

In [14]:
trainX, trainy, testX, testy = utils.load_dataset()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [15]:
verbose, epochs, batch_size = True, 10, 32
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

In [16]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
cce = tf.keras.losses.CategoricalCrossentropy()
cce(y_true, y_pred).numpy()

1.1769392

## Default topology

In [17]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
####
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [20]:
trainy

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/10
230/230 [==============================] - 1s 6ms/step - loss: 0.3971 - accuracy: 0.8432
Epoch 2/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1474 - accuracy: 0.9408
Epoch 3/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1221 - accuracy: 0.9493
Epoch 4/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1115 - accuracy: 0.9523
Epoch 5/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1127 - accuracy: 0.9540
Epoch 6/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1146 - accuracy: 0.9524
Epoch 7/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1028 - accuracy: 0.9566
Epoch 8/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0997 - accuracy: 0.9569
Epoch 9/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0922 - accuracy: 0.9599
Epoch 10/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0813 - accuracy: 0.9621

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 126, 64)           1792      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 124, 64)           12352     
_________________________________________________________________
dropout (Dropout)            (None, 124, 64)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 62, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 3968)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               396900    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6

## Custom Topology

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='tanh', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='tanh'))
model.add(Dropout(0.5))
model.add(AveragePooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='tanh'))
model.add(Dense(n_outputs, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)